## **Collegamento Google Drive**

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# **Valutazione Modelli**

## **Import librerie**

In [4]:
#utils
import numpy as np
import pandas as pd
import seaborn as sns
import joblib
import matplotlib.pyplot as plt

#modelli
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize

## Import dati

In [6]:
#Import dati combinati
data_combined = pd.read_csv('/content/drive/Shareddrives/biometria_progetto/combinated_data.csv')
keystotal = data_combined['Key']
data_combined2 = data_combined.drop(['Key'], axis=1)

#Rimozione dei valori nulli
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
dati_imputed = pd.DataFrame(imputer.fit_transform(data_combined2), columns=data_combined2.columns)

#Normalizzazione dei dati
normalised_data = pd.DataFrame(normalize(dati_imputed, axis = 0))
normalised_data['Key']= keystotal

# **EVALUATE MODEL**

In [9]:
#Valutazione dei modelli
def evaluate_model(X_test, y_test, model_name, model):
    
    y_pred = model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy = accuracy_score(y_test, y_pred)
    # Calcolare la precisione, il richiamo e l'F1-score
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    cm = confusion_matrix(y_test, y_pred)
    
    # Crea un DataFrame con i risultati
    results = pd.DataFrame({'Model': [model_name],
                            'Accuracy': [accuracy],
                            'Precision': [precision],
                            'Recall': [recall],
                            'F1-score': [f1]})
    
    # Salva i risultati in un file CSV
    results.to_csv('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectIndipendent/Metriche_modelli/'+model_name+'_metrics_results.csv', index=False)

    # Calcolare la matrice di confusione normalizzata
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Creare una figura per la matrice di confusione
    plt.figure(figsize=(8, 6))

    # Plotting della matrice di confusione normalizzata utilizzando una heatmap
    sns.heatmap(cm_normalized, annot=True, cmap=plt.cm.Blues)

    # Impostazione dei titoli degli assi
    plt.title(model_name + ' Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    # Salva la matrice di confusione come un file PDF
    plt.savefig('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectIndipendent/Matrici_di_confusione/' + model_name + '_confusion_matrix.pdf', format='pdf', bbox_inches='tight')

    # Chiude la figura
    plt.close()


# **MAIN** (Subject Indipendent con Leave-One-Subject-Out-Cross-Validation)

In [10]:
# Caricamento delle etichette da un file CSV
etichette_train =pd.read_csv('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectIndipendent/etichettetrain.csv')
etichette_test = pd.read_csv('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectIndipendent/etichettetest.csv')

# Lista dei partecipanti
lista_partecipanti = ["ldy", "rx", "qyt", "tyc", "ha", "whh", "mz", "mhw", "zjy", "wll" ,"zjd", "hql", "ly", "wq", "cz"]

## **XGBOOST**

In [16]:
#Gestione del warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.metrics")

In [17]:
# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_xgboost= 0.0
media_xgboost= 0.0
accuracyfinalxgboost= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    X_testd  = data[data['Key'].str.contains(partecipante)]
    X_traind = data[~data['Key'].str.contains(partecipante)]

    X_train = X_traind.drop(['Key'], axis=1)
    X_test =X_testd.drop(['Key'], axis=1)
      
    # Importa il modello migliore di xgboost
    best_xgboost_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_xgboost_model.pkl")
    
    # Salva le metriche di valutazione del modello 
    evaluate_model(X_test, etichette_test, "XGBoost",best_xgboost_model)

    # Addestramento dei con tutti i dati di addestramento
    best_xgboost_model.fit(X_train, np.ravel(etichette_train))

    xgboost_pred = best_xgboost_model.predict(X_test)
    # Calcola le metriche per la classificazione
    accuracy_xgboost = accuracy_score(etichette_test, xgboost_pred)
    
    #Aggiornamento dell'accuratezza totale
    acc_xgboost += accuracy_xgboost
    iterazioni += 1
      
  # Calcolo della media per ogni sessione
  media_xgboost = acc_xgboost / iterazioni

  # Calcolo della media tra le 3 sessioni
  accuracyfinalxgboost += media_xgboost

# Stampa della media dell'accuratezza
print("Media dell'accuratezza XGBOOST:", accuracyfinalxgboost/3)

Media dell'accuratezza XGBOOST: 0.9131172839506174


## **SVM**

In [18]:
# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_svm= 0.0
media_svm= 0.0
accuracyfinalvsm= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    X_testd  = data[data['Key'].str.contains(partecipante)]
    X_traind = data[~data['Key'].str.contains(partecipante)]

    X_train = X_traind.drop(['Key'], axis=1)
    X_test =X_testd.drop(['Key'], axis=1)
      
    # Importa il modello migliore di xgboost
    best_svm_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_svm_model.pkl")

    # Salva le metriche di valutazione del modello 
    evaluate_model(X_test, etichette_test, "SVM", best_svm_model)
  
    # Addestramento dei con tutti i dati di addestramento
    best_svm_model.fit(X_train, np.ravel(etichette_train))
   
    svm_pred= best_svm_model.predict(X_test)
  
    #Calcola le metriche per la classificazione
    accuracy_svm = accuracy_score(etichette_test, svm_pred)

    #Aggiornamento dell'accuratezza totale
    acc_svm += accuracy_svm
    iterazioni += 1
      
  # Calcolo della media per ogni sessione
  media_svm = acc_svm / iterazioni

  #Calcolo della media tra le 3 sessioni
  accuracyfinalvsm += media_svm

# Stampa della media dell'accuratezza
print("Media dell'accuratezza SMV:", accuracyfinalvsm/3)

Media dell'accuratezza SMV: 0.8966049382716049


## **Random Forest**

In [19]:
# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_rf= 0.0
media_rf= 0.0
accuracyfinalrf= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    X_testd  = data[data['Key'].str.contains(partecipante)]
    X_traind = data[~data['Key'].str.contains(partecipante)]

    X_train = X_traind.drop(['Key'], axis=1)
    X_test =X_testd.drop(['Key'], axis=1)
      
    # Importa il modello migliore di xgboost
    best_rf_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_rf_model.pkl")

    # Salva le metriche di valutazione del modello migliore di Random Forest
    evaluate_model(X_test, etichette_test, "RF", best_rf_model)
   
    # Addestramento dei con tutti i dati di addestramento
    best_rf_model.fit(X_train, np.ravel(etichette_train))

    rf_pred=best_rf_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_rf = accuracy_score(etichette_test, rf_pred)
 
    # Aggiornamento dell'accuratezza totale
    acc_rf += accuracy_rf
    iterazioni += 1
      
  # Calcolo della media per ogni sessione
  media_rf = acc_rf / iterazioni

  # Calcolo della media tra le 3 sessioni
  accuracyfinalrf +=media_rf

# Stampa della media dell'accuratezza
print("Media dell'accuratezza Random Forest:", accuracyfinalrf/3)

Media dell'accuratezza Random Forest: 0.9216049382716051


# **Logistic Regression**

In [20]:
# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_lr= 0.0
media_lr= 0.0
accuracyfinallr= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    X_testd  = data[data['Key'].str.contains(partecipante)]
    X_traind = data[~data['Key'].str.contains(partecipante)]

    X_train = X_traind.drop(['Key'], axis=1)
    X_test =X_testd.drop(['Key'], axis=1)
      
    # Importa il modello migliore di xgboost
    best_lr_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_lr_model.pkl")

    # Salva le metriche di valutazione del modello migliore di Logistic Regression
    evaluate_model(X_test, etichette_test, "Logistic Regression", best_lr_model)

    # Addestramento dei con tutti i dati di addestramento
    best_lr_model.fit(X_train,np.ravel(etichette_train))

    lr_pred=best_lr_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_lr = accuracy_score(etichette_test, lr_pred)

    # Aggiornamento dell'accuratezza totale
    acc_lr += accuracy_lr
    iterazioni += 1
      
  # Calcolo della media per ogni sessione
  media_lr = acc_lr / iterazioni

  # Calcolo della media tra le 3 sessioni
  accuracyfinallr +=media_lr

# Stampa della media dell'accuratezza
print("Media dell'accuratezza Logistic Regression:", accuracyfinallr/3)

Media dell'accuratezza Logistic Regression: 0.917283950617284


## **Naive Bayes**

In [21]:
#Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_nb= 0.0
media_nb= 0.0
accuracyfinalnb= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    X_testd  = data[data['Key'].str.contains(partecipante)]
    X_traind = data[~data['Key'].str.contains(partecipante)]

    X_train = X_traind.drop(['Key'], axis=1)
    X_test =X_testd.drop(['Key'], axis=1)
      
    # Importa il modello migliore di xgboost
    best_nb_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_nb_model.pkl")

    # Salva le metriche di valutazione del miglior modello Naive Bayes Classifier
    evaluate_model(X_test, etichette_test, "Naive Bayes", best_nb_model)
   
    # Addestramento dei con tutti i dati di addestramento
    best_nb_model.fit(X_train,np.ravel(etichette_train))

    nb_pred=best_nb_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_nb = accuracy_score(etichette_test, nb_pred)

    # Aggiornamento dell'accuratezza totale
    acc_nb += accuracy_nb
    iterazioni += 1
      
  # Calcolo della media per ogni sessione
  media_nb = acc_nb / iterazioni

  # Calcolo della media tra le 3 sessioni
  accuracyfinalnb +=media_nb

# Stampa della media dell'accuratezza
print("Media dell'accuratezza Naive Bayes:", accuracyfinalnb/3)

Media dell'accuratezza Naive Bayes: 0.8574074074074076


## **AdaBoost**

In [22]:
# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_ab = 0.0
media_ab  =0.0
accuracyfinalab=0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    X_testd  = data[data['Key'].str.contains(partecipante)]
    X_traind = data[~data['Key'].str.contains(partecipante)]

    X_train = X_traind.drop(['Key'], axis=1)
    X_test =X_testd.drop(['Key'], axis=1)
      
    # Importa il modello migliore di xgboost
    best_ab_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_adaboost_model.pkl")

    # Salva le metriche di valutazione del miglior modello Naive Bayes Classifier
    evaluate_model(X_test, etichette_test, "AdaBoostr", best_ab_model)
   
    # Addestramento dei con tutti i dati di addestramento
    best_ab_model.fit(X_train,np.ravel(etichette_train))

    ab_pred=best_ab_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_ab = accuracy_score(etichette_test, nb_pred)
  
    # Aggiornamento dell'accuratezza totale
    acc_ab += accuracy_ab
    iterazioni += 1
      
  # Calcolo della media per ogni sessione
  media_ab = acc_ab / iterazioni

  # Calcolo della media tra le 3 sessioni
  accuracyfinalab +=media_ab

# Stampa della media dell'accuratezza
print("Media dell'accuratezza AdaBoost:", accuracyfinalnb/3)

Media dell'accuratezza AdaBoost: 0.8574074074074076


## **Codice completo con tutti e 6 i modelli**

In [ ]:
'''# Caricamento delle etichette da un file CSV
etichette_train =pd.read_csv('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectIndipendent/etichettetrain.csv')
etichette_test = pd.read_csv('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectIndipendent/etichettetest.csv')

# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_xgboost,acc_svm,acc_rf,acc_lr,acc_nb,acc_ab = 0.0,0.0,0.0,0.0,0.0,0.0
media_xgboost,media_svm,media_rf,media_lr,media_nb,media_ab  =0.0,0.0,0.0,0.0,0.0,0.0
accuracyfinalxgboost,accuracyfinalvsm,accuracyfinalrf,accuracyfinallr,accuracyfinalnb,accuracyfinalab=0.0,0.0,0.0,0.0,0.0,0.0
iterazioni = 0.0

# Lista dei partecipanti
lista_partecipanti = ["ldy", "rx", "qyt", "tyc", "ha", "whh", "mz", "mhw", "zjy", "wll" ,"zjd", "hql", "ly", "wq", "cz"]

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=dati_imputed[dati_imputed['Key'].str.contains(f"{i}_")]
    X_testd  = data[data['Key'].str.contains(partecipante)]
    X_traind = data[~data['Key'].str.contains(partecipante)]

    X_train = X_traind.drop(['Key'], axis=1)
    X_test =X_testd.drop(['Key'], axis=1)
      
    # Importa il modello migliore di xgboost
    best_xgboost_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_xgboost_model.pkl")
    best_svm_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_svm_model.pkl")
    best_rf_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_rf_model.pkl")
    best_lr_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_lr_model.pkl")
    best_nb_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_nb_model.pkl")
    best_ab_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_adaboost_model.pkl")
   
    # Addestramento dei con tutti i dati di addestramento
    best_xgboost_model.fit(X_train, np.ravel(etichette_train))
    best_svm_model.fit(X_train, np.ravel(etichette_train))
    best_rf_model.fit(X_train, np.ravel(etichette_train))
    best_lr_model.fit(X_train,np.ravel(etichette_train))
    best_nb_model.fit(X_train,np.ravel(etichette_train))
    best_ab_model.fit(X_train,np.ravel(etichette_train))

    xgboost_pred = best_xgboost_model.predict(X_test)
    svm_pred= best_svm_model.predict(X_test)
    rf_pred=best_rf_model.predict(X_test)
    lr_pred=best_lr_model.predict(X_test)
    nb_pred=best_nb_model.predict(X_test)
    ab_pred=best_ab_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_xgboost = accuracy_score(etichette_test, xgboost_pred)
    accuracy_svm = accuracy_score(etichette_test, svm_pred)
    accuracy_rf = accuracy_score(etichette_test, rf_pred)
    accuracy_lr = accuracy_score(etichette_test, lr_pred)
    accuracy_nb = accuracy_score(etichette_test, nb_pred)
    accuracy_ab = accuracy_score(etichette_test, nb_pred)
  
    # Aggiornamento dell'accuratezza totale
    acc_xgboost += accuracy_xgboost
    acc_svm += accuracy_svm
    acc_rf += accuracy_rf
    acc_lr += accuracy_lr
    acc_nb += accuracy_nb
    acc_ab += accuracy_ab
    iterazioni += 1
      
  # Calcolo della media per ogni sessione
  media_xgboost = acc_xgboost / iterazioni
  media_svm = acc_svm / iterazioni
  media_rf = acc_rf / iterazioni
  media_lr = acc_lr / iterazioni
  media_nb = acc_nb / iterazioni
  media_ab = acc_ab / iterazioni

  # Calcolo della media tra le 3 sessioni
  accuracyfinalxgboost += media_xgboost
  accuracyfinalvsm += media_svm
  accuracyfinalrf +=media_rf
  accuracyfinallr +=media_lr
  accuracyfinalnb +=media_nb
  accuracyfinalab +=media_ab

# Stampa della media dell'accuratezza
print("Media dell'accuratezza XGBOOST:", accuracyfinalxgboost/3)
print("Media dell'accuratezza SMV:", accuracyfinalvsm/3)
print("Media dell'accuratezza RF:", accuracyfinalrf/3)
print("Media dell'accuratezza LR:", accuracyfinallr/3)
print("Media dell'accuratezza NB:", accuracyfinalnb/3)
print("Media dell'accuratezza AB:", accuracyfinalnb/3)'''

# **Requirements**

In [26]:
#Comando necessario per ottenere le librerie installate insieme alle loro versioni specifiche.
import subprocess
path = '/content/drive/Shareddrives/biometria_progetto/requirements_SI.txt'
subprocess.run(f'pip freeze > {path}', shell=True, check=True)


CompletedProcess(args='pip freeze > /content/drive/Shareddrives/biometria_progetto/requirements_SI.txt', returncode=0)